# Bayesian Temporal Matrix Factorization

**Published**: October 8, 2019

**Revised**: October 8, 2020

**Author**: Xinyu Chen [[**GitHub homepage**](https://github.com/xinychen)]

**Download**: This Jupyter notebook is at our GitHub repository. If you want to evaluate the code, please download the notebook from the [**transdim**](https://github.com/xinychen/transdim/blob/master/imputer/BTMF.ipynb) repository.

This notebook shows how to implement the Bayesian Temporal Matrix Factorization (BTMF), a fully Bayesian matrix factorization model, on some real-world data sets. To overcome the missing data problem in multivariate time series, BTMF takes into account both low-rank matrix structure and time series autoregression. For an in-depth discussion of BTMF, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Xinyu Chen, Lijun Sun (2019). <b>Bayesian temporal factorization for multidimensional time series prediction</b>. arXiv:1910.06366. <a href="https://arxiv.org/pdf/1910.06366.pdf" title="PDF"><b>[PDF]</b></a> 
</font>
</div>

## Problem Description

We assume a spatiotemporal setting for multidimensional time series data throughout this work. In general, modern spatiotemporal data sets collected from sensor networks can be organized as matrix time series. For example, we can denote by matrix $Y\in\mathbb{R}^{N\times T}$ a multivariate time series collected from $N$ locations/sensors on $T$ time points, with each row $$\boldsymbol{y}_{i}=\left(y_{i,1},y_{i,2},...,y_{i,t-1},y_{i,t},y_{i,t+1},...,y_{i,T}\right)$$
corresponding to the time series collected at location $i$.

As mentioned, making accurate predictions on incomplete time series is very challenging, while missing data problem is almost inevitable in real-world applications. Figure 1 illustrates the prediction problem for incomplete time series data. Here we use $(i,t)\in\Omega$ to index the observed entries in matrix $Y$.

<img src="../images/graphical_matrix_time_series.png" alt="drawing" width="500"/>

> **Figure 1**: Illustration of multivariate time series and the prediction problem in the presence of missing values (green: observed data; white: missing data; red: prediction).


In [1]:
import numpy as np
from numpy.linalg import inv as inv
from numpy.random import normal as normrnd
from numpy.random import multivariate_normal as mvnrnd
from scipy.linalg import khatri_rao as kr_prod
from scipy.stats import wishart
from scipy.stats import invwishart
from numpy.linalg import solve as solve
from numpy.linalg import cholesky as cholesky_lower
from scipy.linalg import cholesky as cholesky_upper
from scipy.linalg import solve_triangular as solve_ut
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def mvnrnd_pre(mu, Lambda):
    src = normrnd(size = (mu.shape[0],))
    return solve_ut(cholesky_upper(Lambda, overwrite_a = True, check_finite = False), 
                    src, lower = False, check_finite = False, overwrite_b = True) + mu

def cov_mat(mat, mat_bar):
    mat = mat - mat_bar
    return mat.T @ mat

In [3]:
def sample_factor_w(tau_sparse_mat, tau_ind, W, X, tau, beta0 = 1, vargin = 0):
    """Sampling N-by-R factor matrix W and its hyperparameters (mu_w, Lambda_w)."""
    
    dim1, rank = W.shape
    W_bar = np.mean(W, axis = 0)
    temp = dim1 / (dim1 + beta0)
    var_mu_hyper = temp * W_bar
    var_W_hyper = inv(np.eye(rank) + cov_mat(W, W_bar) + temp * beta0 * np.outer(W_bar, W_bar))
    var_Lambda_hyper = wishart.rvs(df = dim1 + rank, scale = var_W_hyper)
    var_mu_hyper = mvnrnd_pre(var_mu_hyper, (dim1 + beta0) * var_Lambda_hyper)
    
    if dim1 * rank ** 2 > 1e+8:
        vargin = 1
    
    if vargin == 0:
        var1 = X.T
        var2 = kr_prod(var1, var1)
        var3 = (var2 @ tau_ind.T).reshape([rank, rank, dim1]) + var_Lambda_hyper[:, :, None]
        var4 = var1 @ tau_sparse_mat.T + (var_Lambda_hyper @ var_mu_hyper)[:, None]
        for i in range(dim1):
            W[i, :] = mvnrnd_pre(solve(var3[:, :, i], var4[:, i]), var3[:, :, i])
    elif vargin == 1:
        for i in range(dim1):
            pos0 = np.where(sparse_mat[i, :] != 0)
            Xt = X[pos0[0], :]
            var_mu = tau[i] * Xt.T @ sparse_mat[i, pos0[0]] + var_Lambda_hyper @ var_mu_hyper
            var_Lambda = tau[i] * Xt.T @ Xt + var_Lambda_hyper
            W[i, :] = mvnrnd_pre(solve(var_Lambda, var_mu), var_Lambda)
    
    return W

In [4]:
def mnrnd(M, U, V):
    """
    Generate matrix normal distributed random matrix.
    M is a m-by-n matrix, U is a m-by-m matrix, and V is a n-by-n matrix.
    """
    dim1, dim2 = M.shape
    X0 = np.random.randn(dim1, dim2)
    P = cholesky_lower(U)
    Q = cholesky_lower(V)
    
    return M + P @ X0 @ Q.T

def sample_var_coefficient(X, time_lags):
    dim, rank = X.shape
    d = time_lags.shape[0]
    tmax = np.max(time_lags)
    
    Z_mat = X[tmax : dim, :]
    Q_mat = np.zeros((dim - tmax, rank * d))
    for k in range(d):
        Q_mat[:, k * rank : (k + 1) * rank] = X[tmax - time_lags[k] : dim - time_lags[k], :]
    var_Psi0 = np.eye(rank * d) + Q_mat.T @ Q_mat
    var_Psi = inv(var_Psi0)
    var_M = var_Psi @ Q_mat.T @ Z_mat
    var_S = np.eye(rank) + Z_mat.T @ Z_mat - var_M.T @ var_Psi0 @ var_M
    Sigma = invwishart.rvs(df = rank + dim - tmax, scale = var_S)
    
    return mnrnd(var_M, var_Psi, Sigma), Sigma

In [5]:
def sample_factor_x(tau_sparse_mat, tau_ind, time_lags, W, X, A, Lambda_x):
    """Sampling T-by-R factor matrix X."""
    
    dim2, rank = X.shape
    tmax = np.max(time_lags)
    tmin = np.min(time_lags)
    d = time_lags.shape[0]
    A0 = np.dstack([A] * d)
    for k in range(d):
        A0[k * rank : (k + 1) * rank, :, k] = 0
    mat0 = Lambda_x @ A.T
    mat1 = np.einsum('kij, jt -> kit', A.reshape([d, rank, rank]), Lambda_x)
    mat2 = np.einsum('kit, kjt -> ij', mat1, A.reshape([d, rank, rank]))
    
    var1 = W.T
    var2 = kr_prod(var1, var1)
    var3 = (var2 @ tau_ind).reshape([rank, rank, dim2]) + Lambda_x[:, :, None]
    var4 = var1 @ tau_sparse_mat
    for t in range(dim2):
        Mt = np.zeros((rank, rank))
        Nt = np.zeros(rank)
        Qt = mat0 @ X[t - time_lags, :].reshape(rank * d)
        index = list(range(0, d))
        if t >= dim2 - tmax and t < dim2 - tmin:
            index = list(np.where(t + time_lags < dim2))[0]
        elif t < tmax:
            Qt = np.zeros(rank)
            index = list(np.where(t + time_lags >= tmax))[0]
        if t < dim2 - tmin:
            Mt = mat2.copy()
            temp = np.zeros((rank * d, len(index)))
            n = 0
            for k in index:
                temp[:, n] = X[t + time_lags[k] - time_lags, :].reshape(rank * d)
                n += 1
            temp0 = X[t + time_lags[index], :].T - np.einsum('ijk, ik -> jk', A0[:, :, index], temp)
            Nt = np.einsum('kij, jk -> i', mat1[index, :, :], temp0)
        
        var3[:, :, t] = var3[:, :, t] + Mt
        if t < tmax:
            var3[:, :, t] = var3[:, :, t] - Lambda_x + np.eye(rank)
        X[t, :] = mvnrnd_pre(solve(var3[:, :, t], var4[:, t] + Nt + Qt), var3[:, :, t])

    return X

In [6]:
def sample_precision_tau(sparse_mat, mat_hat, ind):
    var_alpha = 1e-6 + 0.5 * np.sum(ind, axis = 1)
    var_beta = 1e-6 + 0.5 * np.sum(((sparse_mat - mat_hat) ** 2) * ind, axis = 1)
    return np.random.gamma(var_alpha, 1 / var_beta)

In [7]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [8]:
def ar4cast(A, X, Sigma, time_lags, multi_step):
    dim, rank = X.shape
    d = time_lags.shape[0]
    X_new = np.append(X, np.zeros((multi_step, rank)), axis = 0)
    for t in range(multi_step):
        var = A.T @ X_new[dim + t - time_lags, :].reshape(rank * d)
        X_new[dim + t, :] = mvnrnd(var, Sigma)
    return X_new

#### BTMF Implementation



In [9]:
def BTMF(dense_mat, sparse_mat, init, rank, time_lags, burn_iter, gibbs_iter, multi_step = 1):
    """Bayesian Temporal Matrix Factorization, BTMF."""
    
    dim1, dim2 = sparse_mat.shape
    d = time_lags.shape[0]
    W = init["W"]
    X = init["X"]
    pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    dense_test = dense_mat[pos_test]
    del dense_mat
    ind = sparse_mat != 0
    pos_obs = np.where(ind)
    tau = np.ones(dim1)
    W_plus = np.zeros((dim1, rank, gibbs_iter))
    A_plus = np.zeros((rank * d, rank, gibbs_iter))
    tau_plus = np.zeros((dim1, gibbs_iter))
    Sigma_plus = np.zeros((rank, rank, gibbs_iter))
    temp_hat = np.zeros(len(pos_test[0]))
    show_iter = 200
    mat_hat_plus = np.zeros((dim1, dim2))
    X_plus = np.zeros((dim2 + multi_step, rank, gibbs_iter))
    mat_new_plus = np.zeros((dim1, multi_step))
    for it in range(burn_iter + gibbs_iter):
        tau_ind = tau[:, None] * ind
        tau_sparse_mat = tau[:, None] * sparse_mat
        W = sample_factor_w(tau_sparse_mat, tau_ind, W, X, tau, beta0 = 1, vargin = 0)
        A, Sigma = sample_var_coefficient(X, time_lags)
        X = sample_factor_x(tau_sparse_mat, tau_ind, time_lags, W, X, A, inv(Sigma))
        mat_hat = W @ X.T
        tau = sample_precision_tau(sparse_mat, mat_hat, ind)
        temp_hat += mat_hat[pos_test]
        if (it + 1) % show_iter == 0 and it < burn_iter:
            temp_hat = temp_hat / show_iter
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, temp_hat)))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, temp_hat)))
            temp_hat = np.zeros(len(pos_test[0]))
            print()
        if it + 1 > burn_iter:
            W_plus[:, :, it - burn_iter] = W
            A_plus[:, :, it - burn_iter] = A
            Sigma_plus[:, :, it - burn_iter] = Sigma
            tau_plus[:, it - burn_iter] = tau
            mat_hat_plus += mat_hat
            X0 = ar4cast(A, X, Sigma, time_lags, multi_step)
            X_plus[:, :, it - burn_iter] = X0
            mat_new_plus += W @ X0[dim2 : dim2 + multi_step, :].T
    mat_hat = mat_hat_plus / gibbs_iter
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_test, mat_hat[:, : dim2][pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_test, mat_hat[:, : dim2][pos_test])))
    print()
    mat_hat = np.append(mat_hat, mat_new_plus / gibbs_iter, axis = 1)
    mat_hat[mat_hat < 0] = 0
    
    return mat_hat, W_plus, X_plus, A_plus, Sigma_plus, tau_plus

In [10]:
def sample_factor_x_partial(tau_sparse_mat, tau_ind, time_lags, W, X, A, Lambda_x, update_horizon):
    """Sampling T-by-R factor matrix X."""
    
    dim2, rank = X.shape
    tmax = np.max(time_lags)
    tmin = np.min(time_lags)
    d = time_lags.shape[0]
    A0 = np.dstack([A] * d)
    for k in range(d):
        A0[k * rank : (k + 1) * rank, :, k] = 0
    mat0 = Lambda_x @ A.T
    mat1 = np.einsum('kij, jt -> kit', A.reshape([d, rank, rank]), Lambda_x)
    mat2 = np.einsum('kit, kjt -> ij', mat1, A.reshape([d, rank, rank]))
    
    var1 = W.T
    var2 = kr_prod(var1, var1)
    var3 = (var2 @ tau_ind).reshape([rank, rank, dim2]) + Lambda_x[:, :, None]
    var4 = var1 @ tau_sparse_mat
    for t in range(dim2 - update_horizon, dim2):
        Mt = np.zeros((rank, rank))
        Nt = np.zeros(rank)
        Qt = mat0 @ X[t - time_lags, :].reshape(rank * d)
        index = list(range(0, d))
        if t >= dim2 - tmax and t < dim2 - tmin:
            index = list(np.where(t + time_lags < dim2))[0]
        if t < dim2 - tmin:
            Mt = mat2.copy()
            temp = np.zeros((rank * d, len(index)))
            n = 0
            for k in index:
                temp[:, n] = X[t + time_lags[k] - time_lags, :].reshape(rank * d)
                n += 1
            temp0 = X[t + time_lags[index], :].T - np.einsum('ijk, ik -> jk', A0[:, :, index], temp)
            Nt = np.einsum('kij, jk -> i', mat1[index, :, :], temp0)
        var3[:, :, t] = var3[:, :, t] + Mt
        X[t, :] = mvnrnd_pre(solve(var3[:, :, t], var4[:, t] + Nt + Qt), var3[:, :, t])

    return X

In [11]:
def BTMF_partial(dense_mat, sparse_mat, init, rank, time_lags, burn_iter, gibbs_iter, multi_step = 1):
    """Bayesian Temporal Matrix Factorization, BTMF."""
    
    dim1, dim2 = sparse_mat.shape
    W_plus = init["W_plus"]
    X_plus = init["X_plus"]
    A_plus = init["A_plus"]
    Sigma_plus = init["Sigma_plus"]
    tau_plus = init["tau_plus"]
    ind = sparse_mat != 0
    pos_obs = np.where(ind)
    show_iter = 200
    X_new_plus = np.zeros((dim2 + multi_step, rank, gibbs_iter))
    mat_new_plus = np.zeros((dim1, multi_step))
    back_step = 10 * multi_step
    for it in range(gibbs_iter):
        tau_ind = tau_plus[:, it][:, None] * ind
        tau_sparse_mat = tau_plus[:, it][:, None] * sparse_mat
        X = sample_factor_x_partial(tau_sparse_mat, tau_ind, time_lags, W_plus[:, :, it], 
                                    X_plus[:, :, it], A_plus[:, :, it], inv(Sigma_plus[:, :, it]), back_step)
        X0 = ar4cast(A_plus[:, :, it], X, Sigma_plus[:, :, it], time_lags, multi_step)
        X_new_plus[:, :, it] = X0
        mat_new_plus += W_plus[:, :, it] @ X0[- multi_step :, :].T
    mat_hat = mat_new_plus / gibbs_iter
    mat_hat[mat_hat < 0] = 0
    
    return mat_hat, W_plus, X_new_plus, A_plus, Sigma_plus, tau_plus

In [12]:
from ipywidgets import IntProgress
from IPython.display import display

def BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter):
    T = dense_mat.shape[1]
    start_time = T - pred_step
    dim1 = dense_mat.shape[0]
    mat_hat = np.zeros((dim1, pred_step))
    
    max_count = int(pred_step / multi_step)
    f = IntProgress(min = 0, max = max_count) # instantiate the bar
    display(f) # display the bar
    for t in range(int(np.ceil(pred_step / multi_step))):
        if t == 0:
            init = {"W": 0.1 * np.random.randn(dim1, rank), "X": 0.1 * np.random.randn(start_time, rank)}
            mat, W, X_new, A, Sigma, tau = BTMF(dense_mat[:, 0 : start_time], 
                sparse_mat[:, 0 : start_time], init, rank, time_lags, burn_iter, gibbs_iter, multi_step)
        else:
            init = {"W_plus": W, "X_plus": X_new, "A_plus": A, "Sigma_plus": Sigma, "tau_plus": tau}
            mat, W, X_new, A, Sigma, tau = BTMF_partial(dense_mat[:, 0 : start_time + t * multi_step], 
                sparse_mat[:, 0 : start_time + t * multi_step], init, rank, time_lags, 
                                                        burn_iter, gibbs_iter, multi_step)
        mat_hat[:, t * multi_step : (t + 1) * multi_step] = mat[:, - multi_step :]
        f.value = t
    small_dense_mat = dense_mat[:, start_time : dense_mat.shape[1]]
    pos = np.where(small_dense_mat != 0)
    print('Prediction MAPE: {:.6}'.format(compute_mape(small_dense_mat[pos], mat_hat[pos])))
    print('Prediction RMSE: {:.6}'.format(compute_rmse(small_dense_mat[pos], mat_hat[pos])))
    print()
    return mat_hat

## Evaluation on Guangzhou Speed Data

**Scenario setting**:

- Tensor size: $214\times 61\times 144$ (road segment, day, time of day)
- Non-random missing (NM)
- 40% missing rate


In [13]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')['random_matrix']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

## Non-random missing (NM)
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 144
- Time lags: {1, 2, 144, 144 + 1, 144 + 2, 7 * 144, 7 * 144 + 1, 7 * 144 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 144
time_lags = np.array([1, 2, 3, 144, 145, 146, 7 * 144, 7 * 144 + 1, 7 * 144 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=504)

Iter: 200
MAPE: 0.103107
RMSE: 4.34289

Iter: 400
MAPE: 0.101911
RMSE: 4.32043

Iter: 600
MAPE: 0.101783
RMSE: 4.3144

Iter: 800
MAPE: 0.101757
RMSE: 4.31506

Iter: 1000
MAPE: 0.101788
RMSE: 4.31628

Imputation MAPE: 0.101783
Imputation RMSE: 4.31732

Prediction MAPE: 0.113524
Prediction RMSE: 4.55913

Running time: 6637 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=252)

Iter: 200
MAPE: 0.102962
RMSE: 4.34077

Iter: 400
MAPE: 0.101998
RMSE: 4.32219

Iter: 600
MAPE: 0.101824
RMSE: 4.31483

Iter: 800
MAPE: 0.101794
RMSE: 4.3132

Iter: 1000
MAPE: 0.101797
RMSE: 4.31296

Imputation MAPE: 0.101828
Imputation RMSE: 4.31437



**Scenario setting**:

- Tensor size: $214\times 61\times 144$ (road segment, day, time of day)
- Random missing (RM)
- 40% missing rate


In [ ]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')['random_tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

## Random missing (RM)
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 144
- Time lags: {1, 2, 144, 144 + 1, 144 + 2, 7 * 144, 7 * 144 + 1, 7 * 144 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 144
time_lags = np.array([1, 2, 3, 144, 145, 146, 7 * 144, 7 * 144 + 1, 7 * 144 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

**Scenario setting**:

- Tensor size: $214\times 61\times 144$ (road segment, day, time of day)
- Random missing (RM)
- 60% missing rate


In [ ]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')['random_tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.6

## Random missing (RM)
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 144
- Time lags: {1, 2, 144, 144 + 1, 144 + 2, 7 * 144, 7 * 144 + 1, 7 * 144 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 144
time_lags = np.array([1, 2, 3, 144, 145, 146, 7 * 144, 7 * 144 + 1, 7 * 144 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

## Evaluation on Hangzhou Flow Data

**Scenario setting**:

- Tensor size: $80\times 25\times 108$ (metro station, day, time of day)
- Non-random missing (NM)
- 40% missing rate


In [ ]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor']
random_matrix = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_matrix.mat')['random_matrix']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

## Non-random missing (NM)
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 108
- Time lags: {1, 2, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 108
time_lags = np.array([1, 2, 3, 108, 109, 110, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

**Scenario setting**:

- Tensor size: $80\times 25\times 108$ (metro station, day, time of day)
- Random missing (RM)
- 40% missing rate


In [ ]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_tensor.mat')['random_tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

## Random missing (RM)
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 108
- Time lags: {1, 2, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 108
time_lags = np.array([1, 2, 3, 108, 109, 110, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

**Scenario setting**:

- Tensor size: $80\times 25\times 108$ (metro station, day, time of day)
- Random missing (RM)
- 60% missing rate


In [ ]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_tensor.mat')['random_tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.6

## Random missing (RM)
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 108
- Time lags: {1, 2, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 108
time_lags = np.array([1, 2, 3, 108, 109, 110, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

## Evaluation on Seattle Speed Data

**Scenario setting**:

- Tensor size: $323\times 28\times 288$ (road segment, day, time of day)
- Non-random missing (NM)
- 40% missing rate


In [ ]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
NM_mat = pd.read_csv('../datasets/Seattle-data-set/NM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
NM_mat = NM_mat.values
missing_rate = 0.4

## Non-random missing (NM)
binary_tensor = np.zeros((dense_mat.shape[0], 28, 288))
for i1 in range(binary_tensor.shape[0]):
    for i2 in range(binary_tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(NM_mat[i1, i2] + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_tensor.reshape([dense_mat.shape[0], dense_mat.shape[1]]))

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 288
- Time lags: {1, 2, 288, 288 + 1, 288 + 2, 7 * 288, 7 * 288 + 1, 7 * 288 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 288
time_lags = np.array([1, 2, 3, 288, 289, 290, 7 * 288, 7 * 288 + 1, 7 * 288 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

**Scenario setting**:

- Tensor size: $323\times 28\times 288$ (road segment, day, time of day)
- Random missing (RM)
- 40% missing rate


In [ ]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
RM_mat = pd.read_csv('../datasets/Seattle-data-set/RM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
RM_mat = RM_mat.values
missing_rate = 0.4

## Random missing (RM)
binary_mat = np.round(RM_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 288
- Time lags: {1, 2, 288, 288 + 1, 288 + 2, 7 * 288, 7 * 288 + 1, 7 * 288 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 288
time_lags = np.array([1, 2, 3, 288, 289, 290, 7 * 288, 7 * 288 + 1, 7 * 288 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

**Scenario setting**:

- Tensor size: $323\times 28\times 288$ (road segment, day, time of day)
- Random missing (RM)
- 60% missing rate


In [ ]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
RM_mat = pd.read_csv('../datasets/Seattle-data-set/RM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
RM_mat = RM_mat.values
missing_rate = 0.6

## Random missing (RM)
binary_mat = np.round(RM_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 288
- Time lags: {1, 2, 288, 288 + 1, 288 + 2, 7 * 288, 7 * 288 + 1, 7 * 288 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 288
time_lags = np.array([1, 2, 3, 288, 289, 290, 7 * 288, 7 * 288 + 1, 7 * 288 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

## Evaluation on London Movement Speed Data

London movement speed data set is is a city-wide hourly traffic speeddataset collected in London.

- Collected from 200,000+ road segments.
- 720 time points in April 2019.
- 73% missing values in the original data.

|  Observation rate | $>90\%$ | $>80\%$ | $>70\%$ | $>60\%$ | $>50\%$ |
|:------------------|--------:|--------:|--------:|--------:|--------:|
|**Number of roads**|  17,666 |  27,148 |  35,912 |  44,352 |  52,727 |


If want to test on the full dataset, you could consider the following setting for masking observations as missing values. 

```python
import numpy as np
np.random.seed(1000)
mask_rate = 0.20

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
pos_obs = np.where(dense_mat != 0)
num = len(pos_obs[0])
sample_ind = np.random.choice(num, size = int(mask_rate * num), replace = False)
sparse_mat = dense_mat.copy()
sparse_mat[pos_obs[0][sample_ind], pos_obs[1][sample_ind]] = 0
```

Notably, you could also consider to evaluate the model on a subset of the data with the following setting.

In [ ]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.4

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Non-random missing (NM)
binary_mat = np.zeros(dense_mat.shape)
random_mat = np.random.rand(dense_mat.shape[0], 30)
for i1 in range(dense_mat.shape[0]):
    for i2 in range(30):
        binary_mat[i1, i2 * 24 : (i2 + 1) * 24] = np.round(random_mat[i1, i2] + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 24
- Time lags: {1, 2, 24, 24 + 1, 24 + 2, 7 * 24, 7 * 24 + 1, 7 * 24 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 24
time_lags = np.array([1, 2, 3, 24, 25, 26, 7 * 24, 7 * 24 + 1, 7 * 24 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

In [ ]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.4

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Random missing (RM)
random_mat = np.random.rand(dense_mat.shape[0], dense_mat.shape[1])
binary_mat = np.round(random_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 24
- Time lags: {1, 2, 24, 24 + 1, 24 + 2, 7 * 24, 7 * 24 + 1, 7 * 24 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 24
time_lags = np.array([1, 2, 3, 24, 25, 26, 7 * 24, 7 * 24 + 1, 7 * 24 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

In [ ]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.6

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Random missing (RM)
random_mat = np.random.rand(dense_mat.shape[0], dense_mat.shape[1])
binary_mat = np.round(random_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 24
- Time lags: {1, 2, 24, 24 + 1, 24 + 2, 7 * 24, 7 * 24 + 1, 7 * 24 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 24
time_lags = np.array([1, 2, 3, 24, 25, 26, 7 * 24, 7 * 24 + 1, 7 * 24 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>